In [1]:
## Importing libraries
import pandas as pd
import numpy as np
import cv2
import os
import glob
from scipy.spatial import distance
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import sys

import tensorflow as tf
from tensorflow.keras import Sequential, models
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPool2D
from keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt
import seaborn as sns

import xmltodict

In [2]:
image_path = "D:\Mask detector\Data\images"
annotations_path = "D:\Mask detector\Data\annotations"


# Data preprocessing


In [3]:
img_names=[] 
xml_names=[] 
for dirname, _, filenames in os.walk('D:\Mask detector\Data'):
    for filename in filenames:
        if os.path.join(dirname, filename)[-3:]!="xml":
            img_names.append(filename)
        else:
            xml_names.append(filename)

img_names[1]
xml_names[1] 

'maksssksksss1.xml'

In [4]:
path_annotations="D:\Mask detector\Data\annotations"
listing=[]
for i in img_names[:]:
    with open(path_annotations+i[:-4]+".xml") as fd:
        doc=xmltodict.parse(fd.read())
    temp=doc["annotation"]["object"]
    if type(temp)==list:
        for i in range(len(temp)):
            listing.append(temp[i]["name"])
    else:
        listing.append(temp["name"])
        

Items = Counter(listing).keys()
values = Counter(listing).values()
print(Items,'\n',values)

OSError: [Errno 22] Invalid argument: 'D:\\Mask detector\\Data\x07nnotationsmaksssksksss0.xml'